# 1. Regression Tree
# 1.1 Dataset preparation

In [10]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_diabetes
def load_dataset():
    '''
    function used for dataset preparation

    Returns:
        X - features of training samples, a pandas dataframe with shape (n, d), where
            X.columns is the column name of X, and we can use X['feat'] to index all
            values of the feature named `feat`.
        y - target values (continuous, scaled) of training samples, a pandas series with shape (n,)
    '''
    X, y = load_diabetes(return_X_y=True)
    X, y = pd.DataFrame(X), pd.Series(y)
    X.columns = ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']
    y = (y - y.mean()) / y.std()
    return X, y
X, y = load_dataset()
print(X.iloc[:5])

        age       sex       bmi        bp        s1        s2        s3  \
0  0.038076  0.050680  0.061696  0.021872 -0.044223 -0.034821 -0.043401   
1 -0.001882 -0.044642 -0.051474 -0.026328 -0.008449 -0.019163  0.074412   
2  0.085299  0.050680  0.044451 -0.005671 -0.045599 -0.034194 -0.032356   
3 -0.089063 -0.044642 -0.011595 -0.036656  0.012191  0.024991 -0.036038   
4  0.005383 -0.044642 -0.036385  0.021872  0.003935  0.015596  0.008142   

         s4        s5        s6  
0 -0.002592  0.019908 -0.017646  
1 -0.039493 -0.068330 -0.092204  
2 -0.002592  0.002864 -0.025930  
3  0.034309  0.022692 -0.009362  
4 -0.002592 -0.031991 -0.046641  


# 1.2 Implement the class of regression tree

In [11]:
### criterion function for regression tree
def sum_squared_distance_to_mean(X, y):
    '''
    function used to calculate the squared error.
    '''
    n = X.shape[0]
    return np.var(y) * n

class DecisionTreeRegressor(object):
    '''
    This class is for decision tree regression

    Attributes:
        - criterion: a function used as the criterion of decision tree regression
        - tolS: a tolerance on the error reduction. we will stop the splitting if
            low error reduction.
        - tree: a nested dictionary representing the decision tree structure.
    '''
    def __init__(self, criterion, tolS=0.1):
        # Initialization
        self.criterion = criterion
        self.tolS = tolS

    def fit(self, X, y):
        '''
        function used to fit the decision tree regressor

        Args:
            X - features of training samples, a pandas dataframe with shape (n, d), where
                X.columns is the column name of X, and we can use X['feat'] to index all
                values of the feature named `feat`.
            y - target values (continuous, scaled) of training samples, a pandas series with shape (n,)
        '''
        self.tree = self.create_tree(X, y)

    def predict(self, X):
        '''
        function used to fit the decision tree regressor

        Args:
            X - features of test samples, a pandas dataframe with shape (n, d)

        Returns:
            y - predictions of test samples, a pandas series with shape (n,)
        '''
        n = X.shape[0]
        y = []
        for i in range(n):
            y.append(self.predict_each(X.iloc[i], self.tree))
        y = pd.Series(y)
        y.index = X.index
        return y

    @staticmethod
    def choose_best_split(X, y, criterion, tolS=1):
        '''
        function used to choose the best split feature and split point

        Args:
            X - features of test samples, a pandas dataframe with shape (n, d)
            y - target values of training samples, a pandas series with shape (n,)
            criterion - function used to measure the quality of a split
            tolS - a tolerance on the error reduction.

        Returns:
            best_feat: the feature used to split on for this node
            best_split: the value of the feature used to split for this node
        '''
        # used to calculate the error reduction
        origin_score = criterion(X, y)
        # initialize
        best_feat, best_split = None, None
        best_score = np.inf
        # search for each feature
        for feat in X.columns:
            # if all values of this feature are equal, do not split this feature
            X_feat_value = set(X[feat])
            if len(X_feat_value) == 1:
                continue
            # otherwise, search for each possible split point of this feature
            for split in X_feat_value:
                # divide the dataset into two parts according to the split
                idx1 = X[feat] < split
                idx2 = X[feat] >= split
                # calculate score to evaluate the quality of a split
                score1 = criterion(X.loc[idx1], y.loc[idx1])
                score2 = criterion(X.loc[idx2], y.loc[idx2])
                score = score1 + score2
                if score < best_score:
                    # choose the split with the largest (variance) reduction
                    best_feat = feat
                    best_split = split
                    best_score = score
        if origin_score - best_score < tolS:
            # Do not split if the (variance) reduction is low
            return None, None
        else:
            # return the feature and the value used for the split
            return best_feat, best_split

    def create_tree(self, X, y):
        '''
        build the decision tree regressor in a recursive manner.
        use a dictionary to represent tree node:
            if the tree node is an internal node, the dictionary will have the following five items:
                - tree["is_leaf"] stores whether the tree node is a leaf node or not.
                - tree["split_feat"] stores the feature used to split on for this node
                - tree["split_point"] stores the value of the feature used to split
                - tree["left"] is a (nested) dictionary used to store the left subtree of this node.
                - tree["right"] is a (nested) dictionary used to store the right subtree of this node.
            if the tree node is a leaf node, the dictionary will have the following two items:
                - tree["is_leaf"] stores whether the tree node is a leaf node or not.
                - tree["value"] stores the prediction at the leaf node
        returns a nested dictionary used to store the tree structure.
        '''
        Tree = {}
        # create a leaf node if all values are equal
        y_values, y_counts = np.unique(y, return_counts=True)
        if len(y_counts) == 1:
            Tree["is_leaf"] = True
            Tree["value"] = y_values[0]
            return Tree
        # create a leaf node if feature set is empty (a low error reduction)
        feat, split = self.choose_best_split(X, y, self.criterion, self.tolS)
        if feat is None:
            Tree["is_leaf"] = True
            Tree["value"] = y.mean()
            return Tree
        # otherwise, create an internal node
        Tree["is_leaf"] = False
        Tree["split_feat"] = feat
        Tree["split_point"] = split
        # divide the dataset (X, y) into two parts according to tree split
        # build the left subtree
        idx = X[feat] < split
        Tree["left"] = self.create_tree(X.loc[idx], y.loc[idx])
        # build the right subtree
        idx = X[feat] >= split
        Tree["right"] = self.create_tree(X.loc[idx], y.loc[idx])
        return Tree

    @staticmethod
    def predict_each(x, tree):
        '''
        for each sample, get the prediction of decision tree regressor in a recursive manner.

        Args:
            x - features of a sample, a pandas Series with shape (d,)
            tree - a nested dictionary representing the decision tree structure.

        Returns:
            the prediction of the sample `x`
        '''
        if tree["is_leaf"] is True:
            # if the `tree` is a leaf node, get the prediction at the leaf node
            return tree["value"]
        else:
            # the 'tree' is a nested dictionary with the following four items:
            #     `split_feat`, `split_point`, `left`, 'right`.
            # get the feature used to split on for this tree
            feat = tree["split_feat"]
            # get the value of the feature used to split
            split = tree["split_point"]
            # get the value of the feature for the sample `x`
            value = x[feat]
            if value < split:
                # search for the left subtree
                return DecisionTreeRegressor.predict_each(x, tree["left"])
            else:
                # search for the right subtree
                return DecisionTreeRegressor.predict_each(x, tree["right"])


## 1.3 Fit the model 

In [12]:
# dataset preparation
X, y = load_dataset()
# initialize the decision tree regressor
model = DecisionTreeRegressor(criterion=sum_squared_distance_to_mean, tolS=0.1)
# fit the regression tree
model.fit(X, y)
# get the prediction of samples
y_hat = model.predict(X)
# calculate the mean squared error of samples
print("The MSE of Random Regression Tree is:", ((y - y_hat)**2).mean())

The MSE of Random Regression Tree is: 0.011446887551729325


# 2. Classification Tree
## 2.1 Dataset preparation

In [13]:
def load_dataset():
    '''
    function used for dataset preparation

    Returns:
        X - features of training samples, a pandas dataframe with shape (n, d), where
            X.columns is the column name of X, and we can use X['feat'] to index all
            values of the feature named `feat`. All features are discrete.
        y - labels of training samples, a pandas series with shape (n,)
    '''
    data = [["green", "curly", "muffled", "clear", "hollow", "hard", "true"],
            ["dark", "curly", "dull", "clear", "hollow", "hard", "true"],
            ["dark", "curly", "muffled", "clear", "hollow", "hard", "true"],
            ["green", "curly", "dull", "clear", "hollow", "hard", "true"],
            ["light", "curly", "muffled", "clear", "hollow", "hard", "true"],
            [
                "green", "slightly curly", "muffled", "clear",
                "slightly hollow", "soft", "true"
            ],
            [
                "dark", "slightly curly", "muffled", "slightly blurry",
                "slightly hollow", "soft", "true"
            ],
            [
                "dark", "slightly curly", "muffled", "clear",
                "slightly hollow", "hard", "true"
            ],
            [
                "dark", "slightly curly", "dull", "slightly blurry",
                "slightly hollow", "hard", "false"
            ],
            ["green", "straight", "crisp", "clear", "flat", "soft", "false"],
            ["light", "straight", "crisp", "blurry", "flat", "hard", "false"],
            ["light", "curly", "muffled", "blurry", "flat", "soft", "false"],
            [
                "green", "slightly curly", "muffled", "slightly blurry",
                "hollow", "hard", "false"
            ],
            [
                "light", "slightly curly", "dull", "slightly blurry", "hollow",
                "hard", "false"
            ],
            [
                "dark", "slightly curly", "muffled", "clear",
                "slightly hollow", "soft", "false"
            ],
            ["light", "curly", "muffled", "blurry", "flat", "hard", "false"],
            [
                "green", "curly", "dull", "slightly blurry", "slightly hollow",
                "hard", "false"
            ]]
    data = pd.DataFrame(data)
    data.columns = [
        "color", "root", "sound", "texture", "umbilicus", "surface", "ripe"
    ]
    X = data.iloc[:, :-1]
    y = data.iloc[:, -1]
    return X, y
X, y = load_dataset()
print(X)

    color            root    sound          texture        umbilicus surface
0   green           curly  muffled            clear           hollow    hard
1    dark           curly     dull            clear           hollow    hard
2    dark           curly  muffled            clear           hollow    hard
3   green           curly     dull            clear           hollow    hard
4   light           curly  muffled            clear           hollow    hard
5   green  slightly curly  muffled            clear  slightly hollow    soft
6    dark  slightly curly  muffled  slightly blurry  slightly hollow    soft
7    dark  slightly curly  muffled            clear  slightly hollow    hard
8    dark  slightly curly     dull  slightly blurry  slightly hollow    hard
9   green        straight    crisp            clear             flat    soft
10  light        straight    crisp           blurry             flat    hard
11  light           curly  muffled           blurry             flat    soft

## 2.2 Criteria

In the following functions, we define the entropy, information gain, gain_ratio, gini and gini index.

### 2.2.1 Information Gain

In [14]:
def entropy(X, y):
    ### calculate the entropy Ent(D) where D = (X, y)
    n = X.shape[0]
    pk = np.unique(y, return_counts=True)[1] / n
    return np.sum(-pk * np.log2(pk))


def information_gain(X, y, feat_values, feat):
    ### calculate the information gain as the guideline for splitting the data set $D = (X, y)$ with feature $feat$.
    n = X.shape[0]
    S = entropy(X, y)
    values = feat_values[feat]
    new_S = 0
    for value in values:
        idx = X[feat] == value
        nv = idx.sum()
        if nv > 0:
            new_S += nv / n * entropy(X.loc[idx], y.loc[idx])
        else:
            # In the calculation of entropy, plog_2(p) = 0 when p = 0.
            new_S += 0
    return S - new_S

### 2.2.2 Gain Ratio

In [15]:
def gain_ratio(X, y, feat_values, feat):
    n = X.shape[0]
    S = entropy(X, y)
    values = feat_values[feat]
    new_S = 0
    iv = 0
    for value in values:
        idx = X[feat] == value
        nv = idx.sum()
        if nv > 0:
            # ignore when nv = 0
            new_S += nv / n * entropy(X.loc[idx], y.loc[idx])
            iv += -(nv / n) * np.log2(nv / n)
    return (S - new_S) / iv

### 2.2.3 Gini Index

In [16]:
def gini(X, y):
    n = X.shape[0]
    pk = np.unique(y, return_counts=True)[1] / n
    return 1 - np.sum(pk**2)


def gini_index(X, y, feat_values, feat):
    n = X.shape[0]
    new_S = 0
    values = feat_values[feat]
    for value in values:
        idx = X[feat] == value
        nv = idx.sum()
        if nv > 0:
            new_S += nv / n * gini(X.loc[idx], y.loc[idx])
    return new_S

## 2.3 Implement the class of classification tree

In [18]:
class DecisionTreeClassifier(object):
    '''
    This class is for classification tree

    Attributes:
        - criterion: a function used as the criterion of classification tree
        - tree: a nested dictionary representing the decision tree structure.
    '''
    
    def __init__(self, criterion):
        # Initialization
        self.criterion = criterion

    def fit(self, X, y):
        n, d = X.shape
        # for each discrete feature, store all possibie values
        self.feat_values = {}
        for feat in X.columns:
            self.feat_values[feat] = X[feat].unique()
        # build the decision tree
        self.tree = self.create_tree(X, y)

    def predict(self, X):
        '''
        function used to fit the decision tree classifier

        Args:
            X - features of test samples, a pandas dataframe with shape (n, d)

        Returns:
            y - predictions of test samples, a pandas series with shape (n,)
        '''
        n = X.shape[0]
        y = []
        for i in range(n):
            y.append(self.predict_each(X.iloc[i], self.tree))
        y = pd.Series(y)
        y.index = X.index
        return y

    @staticmethod
    def choose_best_split(X, y, feat_values, criterion):
        '''
        function used to choose the best split feature

        Args:
            X - features of test samples, a pandas dataframe with shape (n, d)
            y - target values of training samples, a pandas series with shape (n,)
            feat_values - a dict to store all possibie values of each discrete feature
            criterion - function used to measure the quality of a split

        Returns:
            best_feat: the feature used to split on for this node
        '''
        # initialization
        best_feat = None
        best_score = np.inf if criterion == gini_index else -np.inf
        # search for each feature
        for feat in X.columns:
            # if all values of this feature are equal, do not split this feature
            if len(X[feat].unique()) == 1:
                continue
            # otherwise, measure the quality of the split of this discrete feature
            score = criterion(X, y, feat_values, feat)
            if criterion == gini_index:
                # select the feature with the lowest Gini index as the splitting feature
                if score < best_score:
                    best_feat = feat
                    best_score = score
            else:
                # select the feature with the highest information gain / gain ratio
                if score > best_score:
                    best_feat = feat
                    best_score = score
        return best_feat

    def create_tree(self, X, y):
        '''
        build the decision tree classifier in a recursive manner.
        use a dictionary to represent tree node:
            if the tree node is an internal node, the dictionary will have the following items:
                - tree["is_leaf"] stores whether the tree node is a leaf node or not.
                - tree["split_feat"] stores the feature used to split on for this node
                - tree["feat"] is a (nested) dictionary used to store the subtree of this node
                    (i.e., tree["split_feat"] == "feat").
            if the tree node is a leaf node, the dictionary will have the following two items:
                - tree["is_leaf"] stores whether the tree node is a leaf node or not.
                - tree["value"] stores the prediction at the leaf node
        returns a nested dictionary used to store the tree structure.
        '''
        tree = {}
        # create a leaf node if all samples belong to the same class
        classes, class_counts = np.unique(y, return_counts=True)
        if len(classes) == 1:
            tree["is_leaf"] = True
            tree["value"] = classes[0]
            return tree
        # create a leaf node if feature set is empty (i.e., all feature values are the same).
        feat = self.choose_best_split(X, y, self.feat_values, self.criterion)
        if feat is None:
            tree["is_leaf"] = True
            # using the majority vote to get the prediction at the leaf node
            majority_class = classes[np.argmax(class_counts)]
            tree["value"] = majority_class
            return tree
        # otherwise, create an internal node and store the optimal splitting feature `feat`
        tree["is_leaf"] = False
        tree["split_feat"] = feat
        values = self.feat_values[feat]
        # divide the dataset (X, y) into multiple parts according to tree split
        # search for each possible value of the splitting feature, and build child nodes.
        for value in values:
            # calculate the subset of samples whose value of splitting feature is `value`
            idx = X[feat] == value
            if idx.sum() > 0:
                # if there exists sample whose value of splitting feature is `value`,
                # continue to build the subtree.
                tree[value] = self.create_tree(X.loc[idx], y.loc[idx])
            else:
                # Otherwise, mark the child node as a leaf node,
                # and label the child node with the majority class in this tree node.
                majority_class = classes[np.argmax(class_counts)]
                tree[value] = {"is_leaf": True, "value": majority_class}
        return tree

    @staticmethod
    def predict_each(x, tree):
        '''
        for each sample, get the prediction of decision tree classifier in a recursive manner.

        Args:
            x - features of a sample, a pandas Series with shape (d,)
            tree - a nested dictionary representing the decision tree structure.

        Returns:
            the prediction of the sample `x`
        '''
        if tree["is_leaf"] is True:
            # if the `tree` is a leaf node, get the prediction at the leaf node
            return tree["value"]
        else:
            # the 'tree' is a nested dictionary
            # get the value of the feature used to split
            feat = tree["split_feat"]
            # get the value of the feature for the sample `x`
            value = x[feat]
            # search for the corresponding subtree
            return DecisionTreeClassifier.predict_each(x, tree[value])

## 2.4 Fit the model

In [19]:
X, y = load_dataset()

# initialize the decision tree with different criterion
model = DecisionTreeClassifier(criterion=information_gain)
model.fit(X, y)
y_hat = model.predict(X)
print("The accuracy of classfication tree with the information gain is:", (y == y_hat).mean())

model = DecisionTreeClassifier(criterion=gain_ratio)
model.fit(X, y)
y_hat = model.predict(X)
print("The accuracy of classfication tree with the gain ratio is:", (y == y_hat).mean())

model = DecisionTreeClassifier(criterion=gini_index)
model.fit(X, y)
y_hat = model.predict(X)
print("The accuracy of classfication tree with the gini index is:", (y == y_hat).mean())

The accuracy of classfication tree with the information gain is: 1.0
The accuracy of classfication tree with the gain ratio is: 1.0
The accuracy of classfication tree with the gini index is: 1.0
